*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you need to create a machine learning model that will classify SMS messages as either "ham" or "spam". A "ham" message is a normal message sent by a friend. A "spam" message is an advertisement or a message sent by a company.

You should create a function called `predict_message` that takes a message string as an argument and returns a list. The first element in the list should be a number between zero and one that indicates the likeliness of "ham" (0) or "spam" (1). The second element in the list should be the word "ham" or "spam", depending on which is most likely.

For this challenge, you will use the [SMS Spam Collection dataset](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/). The dataset has already been grouped into train data and test data.

The first two cells import the libraries and data. The final cell tests your model and function. Add your code in between these cells.


In [3]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import string
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output
print(tf.__version__)

clear_output()

In [7]:
train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [8]:
train = pd.read_csv(train_file_path, sep='\t', header=None)
test = pd.read_csv(test_file_path, sep='\t', header=None)
train = train.rename(columns={0: "class", 1: "message"})
test = test.rename(columns={0: "class", 1: "message"})

train['class'] = pd.factorize(train['class'])[0]
test['class'] = pd.factorize(test['class'])[0]

In [9]:
def remove_punct(text):
  translator = str.maketrans("", "", string.punctuation)
  return text.translate(translator)

In [10]:
train.message = train.message.map(remove_punct)
test.message = test.message.map(remove_punct)

In [11]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stop = set(stopwords.words("english"))

def remove_stopwords(text):
  filtered_words = [word.lower() for word in text.split() if word.lower() not in stop]
  return " ".join(filtered_words)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\alex\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [12]:
train.message = train.message.map(remove_stopwords)
test.message = test.message.map(remove_stopwords)

In [13]:
from collections import Counter

def counter_word(text_col):
  count = Counter()
  for text in text_col.values:
    print(text)
    for word in text.split():
      count[word] += 1
  return count

counter = counter_word(train.message) + counter_word(test.message)
clear_output()

In [14]:
num_unique_words = len(counter)

In [15]:
train_message = train.message.to_numpy()
train_labels = train['class'].to_numpy()
test_message = test.message.to_numpy()
test_labels = test['class'].to_numpy()

In [16]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=num_unique_words)
tokenizer.fit_on_texts(train_message)


In [17]:
word_index = tokenizer.word_index

In [18]:
train_sequences = tokenizer.texts_to_sequences(train_message)
test_sequences = tokenizer.texts_to_sequences(test_message)

In [19]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

max_length = 20

train_padded = pad_sequences(train_sequences, maxlen=max_length, padding="post", truncating="post")
test_padded = pad_sequences(test_sequences, maxlen=max_length, padding="post", truncating="post")

In [20]:
from tensorflow.keras import layers

model = keras.models.Sequential()
model.add(layers.Embedding(num_unique_words, 32, input_length=max_length))
model.add(layers.LSTM(32, dropout=0.1))
model.add(layers.Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 32)            304640    
                                                                 
 lstm (LSTM)                 (None, 32)                8320      
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 312,993
Trainable params: 312,993
Non-trainable params: 0
_________________________________________________________________


In [21]:
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['acc'])

In [22]:
model.fit(train_padded, train_labels, epochs=15, validation_data=(test_padded, test_labels), validation_split=0.2, verbose=2)

Epoch 1/15
131/131 - 5s - loss: 0.1944 - acc: 0.9354 - val_loss: 0.0891 - val_acc: 0.9792 - 5s/epoch - 40ms/step
Epoch 2/15
131/131 - 1s - loss: 0.0591 - acc: 0.9840 - val_loss: 0.0678 - val_acc: 0.9828 - 947ms/epoch - 7ms/step
Epoch 3/15
131/131 - 1s - loss: 0.0321 - acc: 0.9916 - val_loss: 0.0599 - val_acc: 0.9849 - 936ms/epoch - 7ms/step
Epoch 4/15
131/131 - 1s - loss: 0.0234 - acc: 0.9935 - val_loss: 0.0446 - val_acc: 0.9849 - 912ms/epoch - 7ms/step
Epoch 5/15
131/131 - 1s - loss: 0.0163 - acc: 0.9955 - val_loss: 0.0467 - val_acc: 0.9892 - 954ms/epoch - 7ms/step
Epoch 6/15
131/131 - 1s - loss: 0.0133 - acc: 0.9962 - val_loss: 0.0580 - val_acc: 0.9864 - 910ms/epoch - 7ms/step
Epoch 7/15
131/131 - 1s - loss: 0.0091 - acc: 0.9981 - val_loss: 0.0851 - val_acc: 0.9835 - 919ms/epoch - 7ms/step
Epoch 8/15
131/131 - 1s - loss: 0.0056 - acc: 0.9988 - val_loss: 0.0954 - val_acc: 0.9842 - 903ms/epoch - 7ms/step
Epoch 9/15
131/131 - 1s - loss: 0.0066 - acc: 0.9986 - val_loss: 0.0828 - val_acc:

In [23]:
def encode_text(text):
    tokens = keras.preprocessing.text.text_to_word_sequence(text)
    tokens = [word_index[word] if word in word_index else 0 for word in tokens]
    return pad_sequences([tokens], maxlen=max_length, padding="post", truncating="post")[0]

In [24]:
def predict_message(pred_text):
  encoded_text = remove_punct(pred_text)
  encoded_text = encode_text(pred_text)
  pred = np.zeros((1,20))
  pred[0] = encoded_text
  result = model.predict(pred, verbose=0)
  prediction = [1 if result[0] > 0.5 else 0]
  category = 'spam' if prediction[0] == 1 else 'ham'
  prediction.append(category)
  return prediction

pred_text = "sale today! to stop texts call 98912460324"

prediction = predict_message(pred_text)
print(prediction)

[1, 'spam']


In [25]:
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


You passed the challenge. Great job!
